In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()
model.load_state_dict(torch.load('resnet18_finetuned_fp16_pruned.pth'))
device = torch.device('cuda')
print("模型成功加载")

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 134, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

In [ ]:
torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt.pth')

# 預測


## 載入TRT 循線resnet

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))

<All keys matched successfully>

### 設定欲處理圖片方法

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
device = torch.device('cuda')
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

#### 開啟相機

In [4]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

RuntimeError: Could not initialize camera.  Please see error trace.

#### 創建顯示模組


In [ ]:
import ipywidgets.widgets as widgets
import ipywidgets as widgets
from IPython.display import display

image_widget = ipywidgets.Image()
label = widgets.Label(value="Waiting for data...")
display(image_widget,label)

#### 開啟yolov4-tiny-trt

In [ ]:
import os
import time

import pycuda.autoinit
from utils.yolo_classes import get_cls_dict
from utils.display import open_window, set_display, show_fps
from utils.visualization import BBoxVisualization
from utils.yolo import TRT_YOLO
trt_yolo = TRT_YOLO("yolov4-tiny-224", (224, 224), 4)
def getNearest(signs):
    # Find the index of the sign with the largest width
    largest_sign = max(signs, key=lambda x: x)
    # Assuming that the signs array is paired with (width, class_id), return the sign data (e.g., class id and width)
    return largest_sign

#### 設定路牌


In [ ]:
SIGN_INDEX_REVERSED = {
    0: 'Maximum Speed Limit',
    1: 'Minimum Speed Limit',
    2: 'Pedestrian Crossing',
    3: 'Railroad Crossing Warning',
    4: 'Right Turn',
    5: 'Stop',
    6: 'Stop and Proceed'
}

##

In [ ]:
def execute(change):
    global angle, angle_last
    image = change['new']
    height = image.shape[0]
    
    # 裁剪影像
    cropped_image = image[int(height * 0.4):, :, :]
    
    # 模型推理
    xy = model_trt(preprocess(cropped_image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = xy[1]
    widget_width,widget_height=224,224
    x_pixel =int( x* widget_width / 2 + widget_width / 2)
    y_pixel =int(y * widget_height / 2 + widget_height / 2)
    
    # 在圖片上繪製預測結果
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # 確保格式正確
    cv2.circle(image, (x_pixel, y_pixel), 5, (0, 255, 0), -1)  # 綠色點表示預測位置
    
    
    #### YOLO
    boxes, confs, clss = trt_yolo.detect(image)
    signs = []
    for box, cls in zip(boxes, clss):
        width = box[2] - box[0]
        signs.append([width, cls])
        x1, y1, x2, y2 = map(int, box)
        label1 = SIGN_INDEX_REVERSED[int(cls)]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        cv2.putText(image, label1, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    # 更新 widget 顯示處理後的圖片
    image_widget.value = bgr8_to_jpeg(image)
    # 更新標籤
    label.value = f"X: {x:.2f}, Y: {y:.2f}\n"+f"signs:{signs}"

# 測試輸出
execute({'new': camera.value})

In [ ]:
while(1):
    execute({'new': camera.value})   

In [5]:
camera.stop()

NameError: name 'camera' is not defined